In [1]:
import logging
import imp
imp.reload(logging)
logging.basicConfig(level=logging.INFO)

In [2]:
import gensim.models.doc2vec as d2v
import nltk

INFO:summa.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


INFO:gensim.utils:Pattern library is not installed, lemmatization won't be available.


In [3]:
import psycopg2 as db
conn = db.connect('dbname=TBM')
cur = conn.cursor()

In [4]:
def preprocess(article):
    article = article.lower()
    # article = '__BEG_ARTICLE__ ' + article
    # article = article + ' __END_ARTICLE__'
    # article = article.replace('\n\n',' __PARTOKEN__ ')
    article = article.replace('u.s.','usa')
    article = article.replace('.com',' __dotcom__')
    article = article.replace("doesn't",'does not')
    article = article.replace("don't",'do not')
    article = article.replace('los angeles','losangeles')
    article = article.replace('new-york','newyork')
    article = article.replace('new york','newyork')
    article = article.replace("'s",'')
    article = article.replace("\'",' ')
    article = article.replace('\"',' ')
    article = article.replace('-',' ')
    article = article.replace('.',' ')
    article = article.replace(',',' ')
    article = article.replace('(',' ')
    article = article.replace(')',' ')
    article = article.replace('/',' ')
    article = article.replace(';',' ')
    article = article.replace(':',' ')
    article = article.replace(' to ',' ')
    article = article.replace(' in ',' ')
    article = article.replace(' for ',' ')
    article = article.replace(' on ',' ')
    article = article.replace(' as ',' ')
    article = article.replace(' of ',' ')
    article = article.replace(' by ',' ')
    article = article.replace(' at ',' ')
    article = article.replace('*',' ')
    article = article.replace('$',' $ ')
    article = article.replace('®','')
        
    words = nltk.word_tokenize(article)
    return words                   

In [14]:
class Articles:
    def __init__(self,n='ALL'):
        if isinstance(n,int):
            n = str(n)
        select_query = 'SELECT id,content FROM articles3 LIMIT %s' % n
        self.batch_size = 1000
        cur.execute(select_query)
        self.records = []
        
    def __iter__(self):
        return self

    def __next__(self):
        if not self.records:
            self.records = cur.fetchmany(self.batch_size)
            if not self.records:
                raise StopIteration()
        next_article = self.records.pop(0)
        return { 'id': str(next_article[0]), 'content': next_article[1] }

    def __len__(self):
        i = 0
        while True:
            try:
                self.__next__()
                i = i+1
            except:
                break
        return i


class TaggedArticles:
    def __init__(self,n='ALL'):
        self.n = n
        # self.articles = Articles(n)

    def __iter__(self):
        self._reset()
        return self

    def __next__(self):
        next_article = self.articles.__next__()
        content = next_article['content']
        id = next_article['id']
        return d2v.TaggedDocument(words=preprocess(content),tags=[str(id)])

    def _reset(self):
        self.articles = Articles(self.n)

    def __len__(self):
        return len(self.articles)

In [20]:
m = d2v.Doc2Vec(TaggedArticles(),workers=8)

INFO:gensim.models.word2vec:training on 158718280 raw words (157009330 effective words) took 1217.2s, 128994 effective words/s


INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 0 more threads


INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 1 more threads


INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 2 more threads


INFO:gensim.models.word2vec:worker thread finished; awaiting finish of 3 more threads


...

INFO:gensim.models.doc2vec:PROGRESS: at example #30000, processed 14574557 words (150324/s), 149018 word types, 30000 tags


INFO:gensim.models.doc2vec:PROGRESS: at example #20000, processed 10014266 words (150627/s), 119981 word types, 20000 tags


INFO:gensim.models.doc2vec:PROGRESS: at example #10000, processed 5257626 words (142599/s), 77742 word types, 10000 tags


INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


INFO:gensim.models.doc2vec:collecting all words and their counts


In [32]:
m.most_similar('')

[('wildcats', 0.4862581491470337),
 ('usa”', 0.4291471242904663),
 ('sidelines', 0.42285704612731934),
 ('remainder', 0.42167505621910095),
 ('playoffs', 0.41988569498062134),
 ('buckeyes', 0.41739606857299805),
 ('truth', 0.41589754819869995),
 ('scene', 0.4122736155986786),
 ('rest', 0.4103614091873169),
 ('latter', 0.4053933620452881)]